In [1]:
import os
import shutil
import sys
import glob
import pickle
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split

from pathlib import Path

In [2]:
SEED = 42
if SEED is not None:
    np.random.seed(SEED)
    random.seed(SEED)
    tf.random.set_random_seed(SEED)

In [3]:
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

## Чтение данных

In [4]:
DATA_PATH = Path('data') / 'criteo-part'

TRAIN_PATH = str(DATA_PATH / 'train.csv')

In [5]:
train = pd.read_csv(TRAIN_PATH)

In [6]:
train.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [7]:
train = train.rename(columns=lambda x: x.replace('_', '').upper())

In [8]:
train.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C31,C32,C33,C34,C35,C36,C37,C38,C39,ID
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [9]:
num_columns = ['C{}'.format(i) for i in range(1, 14)]
cat_columns = ['C{}'.format(i) for i in range(14, 40)]
target = ['C0']
len(num_columns), len(cat_columns)

(13, 26)

In [10]:
X, y = train.drop(columns=['C0', 'ID']), train['C0']

# Data preprocessing

In [11]:
X[num_columns] = X[num_columns].fillna(0)
X[cat_columns] = X[cat_columns].fillna('-1')

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
X_encoded = X.copy()

In [14]:
mms = MinMaxScaler(feature_range=(0, 1))

In [15]:
X_encoded[num_columns] = mms.fit_transform(X_encoded[num_columns])

In [16]:
X_train_encoded, X_valid_encoded, y_train, y_valid = train_test_split(X_encoded, y, stratify=y, random_state=SEED)

In [17]:
X_train_encoded.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39
597429,0.000000,0.033123,0.000427,0.000000,5.553482e-04,0.000669,0.000202,0.004752,0.001966,0.000000,...,07c540c4,561cabfe,21ddcdc9,5840adea,a466b525,-1,32c7478e,9b18ad04,7a402766,c338ef12
1768963,0.000000,0.000136,0.000412,0.021665,5.553482e-04,0.002609,0.000101,0.005741,0.004450,0.000000,...,d4bb7bd8,13145934,55dd3565,a458ea53,07256a82,-1,3a171ecb,75c8ca05,f0f449dd,d21d0b82
1913948,0.000000,0.000861,0.000137,0.007982,8.473158e-04,0.000129,0.003737,0.001386,0.002949,0.000000,...,e5ba7672,cc0b0790,712d530c,b1252a9d,9453aa3a,-1,c7dc6720,7e99499d,010f6491,bd422f81
266546,0.000000,0.000181,0.000076,0.007982,2.668111e-06,0.000000,0.000000,0.001782,0.000362,0.000000,...,1e88c74f,6fc84bfb,-1,-1,4f1aa25f,c9d4222a,be7c41b4,ded4aac9,-1,-1
800178,0.000519,0.000181,0.000031,0.000000,7.623174e-07,0.000000,0.000303,0.000198,0.000052,0.222222,...,e5ba7672,7ef5affa,3014a4b1,b1252a9d,f8ffd97a,-1,32c7478e,08abcea9,001f3601,50f8fc74


In [18]:
# sparse_features = [SparseFeat(feat, X_encoded[feat].nunique() + 1) for feat in cat_columns]
# or using hashing
sparse_features = [SparseFeat(feat, vocabulary_size=1000, embedding_dim=4, use_hash=True, dtype='string')  for feat in cat_columns]
dense_features = [DenseFeat(feat, 1) for feat in num_columns]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
fixlen_feature_columns = sparse_features + dense_features

In [20]:
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [21]:
train_model_input = {name:X_train_encoded[name] for name in feature_names}

In [22]:
test_model_input = {name:X_valid_encoded[name] for name in feature_names}

# Model training

In [23]:
from deepctr.models import DeepFM, CCPM, PNN, WDL, MLR, NFM, AFM, DCN, xDeepFM, AutoInt, ONN, FiBiNET
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, log_loss

In [24]:
def get_metrics(groups) -> pd.DataFrame:
    metrics =  pd.DataFrame.from_records(groups, columns=groups.keys(), index=list(all_metrics.values())[0].keys())
    metrics.index.name = 'metric'
    return metrics

def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    metrics = get_metrics(groups)
    return metrics.subtract(metrics[control_name], axis='index').drop(columns=control_name) * 100


all_metrics = {}

In [25]:
def test_model(model, train_model_input, test_model_input, y_train, y_valid, batch_size=4096 * 2, model_name='model'):
    checkpoint_path = f'models/criteo/{model_name}'
    os.makedirs(checkpoint_path, exist_ok=True)
    checkpoint_path += '/model.ckpt'
    
    model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", tf.keras.metrics.AUC()])
    
    es = EarlyStopping(monitor='val_auc', verbose=1, patience=2, mode='max')
    mdckpt = ModelCheckpoint(filepath=checkpoint_path, monitor='val_auc', mode='max', save_best_only=True, save_weights_only=True)
    
    history = model.fit(train_model_input, y_train,
                        batch_size=batch_size, epochs=15, verbose=2, validation_split=0.2, callbacks=[es, mdckpt])
    
    model.load_weights(checkpoint_path)
    
    y_valid_predicted = model.predict(test_model_input, batch_size=batch_size).squeeze()
    
    scores = {
        'roc-auc': roc_auc_score(y_valid, y_valid_predicted),
        'log-loss': log_loss(y_valid, y_valid_predicted)        
    }
    del model
    tf.keras.backend.clear_session()
    return scores

## CCPM (Convolutional Click Prediction Model)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/CCPM.png" alt="drawing" width="900"/>

In [26]:
ccpm_model = CCPM(linear_feature_columns, sparse_features, dnn_dropout=0.5)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [27]:
all_metrics['CCPM'] = test_model(ccpm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='CCPM')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 115s - loss: 0.5258 - binary_crossentropy: 0.5253 - auc: 0.6881 - val_loss: 0.4997 - val_binary_crossentropy: 0.4989 - val_auc: 0.7366
Epoch 2/15
2198958/2198958 - 111s - loss: 0.4949 - binary_crossentropy: 0.4938 - auc: 0.7430 - val_loss: 0.4962 - val_binary_crossentropy: 0.4951 - val_auc: 0.7422
Epoch 3/15
2198958/2198958 - 112s - loss: 0.4909 - binary_crossentropy: 0.4896 - auc: 0.7493 - val_loss: 0.4948 - val_binary_crossentropy: 0.4934 - val_auc: 0.7449
Epoch 4/15
2198958/2198958 - 112s - loss: 0.4880 - binary_crossentropy: 0.4865 - auc: 0.7538 - val_loss: 0.4936 - val_binary_crossentropy: 0.4920 - val_auc: 0.7472
Epoch 5/15
2198958/2198958 - 113s - loss: 0.4853 - binary_crossentropy: 0.4836 - auc: 0.7579 - val_loss: 0.4927 - val_binary_crossentropy: 0.4908 - val_auc: 0.7487
Epoch 6/15
2198958/2198958 - 111s

## PNN (Product-based Neural Network)
<!-- ![pic](https://deepctr-torch.readthedocs.io/en/latest/_images/PNN.png) -->
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/PNN.png" alt="drawing" width="900"/>

In [28]:
pnn_model = PNN(dnn_feature_columns, dnn_dropout=0.5)

In [29]:
all_metrics['PNN'] = test_model(pnn_model, train_model_input, test_model_input, y_train.values, y_valid,  model_name='PNN')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 27s - loss: 0.5152 - binary_crossentropy: 0.5149 - auc: 0.7082 - val_loss: 0.4863 - val_binary_crossentropy: 0.4859 - val_auc: 0.7568
Epoch 2/15
2198958/2198958 - 26s - loss: 0.4830 - binary_crossentropy: 0.4825 - auc: 0.7608 - val_loss: 0.4790 - val_binary_crossentropy: 0.4784 - val_auc: 0.7673
Epoch 3/15
2198958/2198958 - 26s - loss: 0.4764 - binary_crossentropy: 0.4756 - auc: 0.7701 - val_loss: 0.4762 - val_binary_crossentropy: 0.4754 - val_auc: 0.7715
Epoch 4/15
2198958/2198958 - 26s - loss: 0.4719 - binary_crossentropy: 0.4710 - auc: 0.7763 - val_loss: 0.4753 - val_binary_crossentropy: 0.4742 - val_auc: 0.7733
Epoch 5/15
2198958/2198958 - 26s - loss: 0.4686 - binary_crossentropy: 0.4674 - auc: 0.7808 - val_loss: 0.4745 - val_binary_crossentropy: 0.4733 - val_auc: 0.7739
Epoch 6/15
2198958/2198958 - 26s - loss: 0.4659 - binary_crossentropy: 0.4646 - auc: 0.7842 - val_loss: 0.4751 - val_binary_crossent

## Wide & Deep
<!-- ![pic](https://deepctr-torch.readthedocs.io/en/latest/_images/WDL.png) -->
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/WDL.png" alt="drawing" width="900"/>

In [30]:
wdl_model = WDL(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [31]:
all_metrics['WDL'] = test_model(wdl_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='WDL')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 25s - loss: 0.5096 - binary_crossentropy: 0.5093 - auc: 0.7176 - val_loss: 0.4855 - val_binary_crossentropy: 0.4851 - val_auc: 0.7580
Epoch 2/15
2198958/2198958 - 24s - loss: 0.4824 - binary_crossentropy: 0.4818 - auc: 0.7619 - val_loss: 0.4802 - val_binary_crossentropy: 0.4795 - val_auc: 0.7657
Epoch 3/15
2198958/2198958 - 24s - loss: 0.4776 - binary_crossentropy: 0.4768 - auc: 0.7685 - val_loss: 0.4778 - val_binary_crossentropy: 0.4769 - val_auc: 0.7692
Epoch 4/15
2198958/2198958 - 24s - loss: 0.4744 - binary_crossentropy: 0.4734 - auc: 0.7731 - val_loss: 0.4766 - val_binary_crossentropy: 0.4755 - val_auc: 0.7711
Epoch 5/15
2198958/2198958 - 25s - loss: 0.4719 - binary_crossentropy: 0.4708 - auc: 0.7764 - val_loss: 0.4760 - val_binary_crossentropy: 0.4748 - val_auc: 0.7723
Epoch 6/15
2198958/2198958 - 24s - loss: 0.4700 - binary_crossentropy: 0.4686 - auc: 0.7791 - val_loss: 0.4758 - val_binary_crossent

## DeepFM
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/DeepFM.png" alt="drawing" width="900"/>

In [32]:
deep_fm_model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [33]:
all_metrics['DeepFM'] = test_model(deep_fm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='DeepFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 26s - loss: 0.5095 - binary_crossentropy: 0.5092 - auc: 0.7176 - val_loss: 0.4854 - val_binary_crossentropy: 0.4850 - val_auc: 0.7581
Epoch 2/15
2198958/2198958 - 24s - loss: 0.4822 - binary_crossentropy: 0.4816 - auc: 0.7620 - val_loss: 0.4802 - val_binary_crossentropy: 0.4795 - val_auc: 0.7658
Epoch 3/15
2198958/2198958 - 24s - loss: 0.4772 - binary_crossentropy: 0.4764 - auc: 0.7689 - val_loss: 0.4779 - val_binary_crossentropy: 0.4770 - val_auc: 0.7691
Epoch 4/15
2198958/2198958 - 24s - loss: 0.4742 - binary_crossentropy: 0.4732 - auc: 0.7732 - val_loss: 0.4766 - val_binary_crossentropy: 0.4755 - val_auc: 0.7709
Epoch 5/15
2198958/2198958 - 25s - loss: 0.4718 - binary_crossentropy: 0.4706 - auc: 0.7767 - val_loss: 0.4763 - val_binary_crossentropy: 0.4751 - val_auc: 0.7726
Epoch 6/15
2198958/2198958 - 25s - loss: 0.4699 - binary_crossentropy: 0.4685 - auc: 0.7793 - val_loss: 0.4751 - val_binary_crossent

## MLR(Mixed Logistic Regression/Piece-wise Linear Model)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/MLR.png" alt="drawing" width="900"/>

In [34]:
mlr_model = MLR(linear_feature_columns, dnn_feature_columns)

In [35]:
all_metrics['MLR'] = test_model(mlr_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='MLR')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 31s - loss: 0.5371 - binary_crossentropy: 0.5359 - auc: 0.6695 - val_loss: 0.5047 - val_binary_crossentropy: 0.5026 - val_auc: 0.7321
Epoch 2/15
2198958/2198958 - 28s - loss: 0.4976 - binary_crossentropy: 0.4950 - auc: 0.7420 - val_loss: 0.4979 - val_binary_crossentropy: 0.4950 - val_auc: 0.7427
Epoch 3/15
2198958/2198958 - 28s - loss: 0.4929 - binary_crossentropy: 0.4898 - auc: 0.7496 - val_loss: 0.4958 - val_binary_crossentropy: 0.4926 - val_auc: 0.7464
Epoch 4/15
2198958/2198958 - 28s - loss: 0.4908 - binary_crossentropy: 0.4874 - auc: 0.7531 - val_loss: 0.4945 - val_binary_crossentropy: 0.4911 - val_auc: 0.7487
Epoch 5/15
2198958/2198958 - 28s - loss: 0.4895 - binary_crossentropy: 0.4859 - auc: 0.7553 - val_loss: 0.4937 - val_binary_crossentropy: 0.4901 - val_auc: 0.7500
Epoch 6/15
2198958/2198958 - 27s - loss: 0.4886 - binary_crossentropy: 0.4849 - auc: 0.7568 - val_loss: 0.4931 - val_binary_crossent

## NFM (Neural Factorization Machine)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/NFM.png" alt="drawing" width="900"/>

In [36]:
nfm_model = NFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [37]:
all_metrics['NFM'] = test_model(nfm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='NFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 26s - loss: 0.5117 - binary_crossentropy: 0.5114 - auc: 0.7144 - val_loss: 0.4863 - val_binary_crossentropy: 0.4857 - val_auc: 0.7577
Epoch 2/15
2198958/2198958 - 25s - loss: 0.4825 - binary_crossentropy: 0.4817 - auc: 0.7622 - val_loss: 0.4816 - val_binary_crossentropy: 0.4808 - val_auc: 0.7650
Epoch 3/15
2198958/2198958 - 24s - loss: 0.4769 - binary_crossentropy: 0.4759 - auc: 0.7698 - val_loss: 0.4790 - val_binary_crossentropy: 0.4779 - val_auc: 0.7682
Epoch 4/15
2198958/2198958 - 24s - loss: 0.4727 - binary_crossentropy: 0.4715 - auc: 0.7754 - val_loss: 0.4779 - val_binary_crossentropy: 0.4766 - val_auc: 0.7696
Epoch 5/15
2198958/2198958 - 24s - loss: 0.4696 - binary_crossentropy: 0.4681 - auc: 0.7796 - val_loss: 0.4781 - val_binary_crossentropy: 0.4765 - val_auc: 0.7699
Epoch 6/15
2198958/2198958 - 24s - loss: 0.4673 - binary_crossentropy: 0.4657 - auc: 0.7827 - val_loss: 0.4781 - val_binary_crossent

## AFM (Attentional Factorization Machine)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/AFM.png" alt="drawing" width="900"/>

In [38]:
afm_model = AFM(linear_feature_columns, sparse_features, afm_dropout=0.5)

Instructions for updating:
dim is deprecated, use axis instead


In [39]:
all_metrics['AFM'] = test_model(afm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='AFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 33s - loss: 0.5468 - binary_crossentropy: 0.5465 - auc: 0.6482 - val_loss: 0.5195 - val_binary_crossentropy: 0.5189 - val_auc: 0.7114
Epoch 2/15
2198958/2198958 - 32s - loss: 0.5101 - binary_crossentropy: 0.5091 - auc: 0.7236 - val_loss: 0.5069 - val_binary_crossentropy: 0.5057 - val_auc: 0.7273
Epoch 3/15
2198958/2198958 - 32s - loss: 0.5020 - binary_crossentropy: 0.5006 - auc: 0.7343 - val_loss: 0.5031 - val_binary_crossentropy: 0.5015 - val_auc: 0.7334
Epoch 4/15
2198958/2198958 - 32s - loss: 0.4987 - binary_crossentropy: 0.4969 - auc: 0.7397 - val_loss: 0.5011 - val_binary_crossentropy: 0.4991 - val_auc: 0.7368
Epoch 5/15
2198958/2198958 - 32s - loss: 0.4966 - binary_crossentropy: 0.4945 - auc: 0.7432 - val_loss: 0.4997 - val_binary_crossentropy: 0.4975 - val_auc: 0.7392
Epoch 6/15
2198958/2198958 - 32s - loss: 0.4950 - binary_crossentropy: 0.4927 - auc: 0.7458 - val_loss: 0.4987 - val_binary_crossent

## DCN (Deep & Cross Network)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/DCN.png" alt="drawing" width="900"/>

In [40]:
dcn_model = DCN(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [41]:
all_metrics['DCN'] = test_model(dcn_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='DCN')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 26s - loss: 0.5085 - binary_crossentropy: 0.5082 - auc: 0.7197 - val_loss: 0.4857 - val_binary_crossentropy: 0.4852 - val_auc: 0.7581
Epoch 2/15
2198958/2198958 - 26s - loss: 0.4823 - binary_crossentropy: 0.4817 - auc: 0.7620 - val_loss: 0.4807 - val_binary_crossentropy: 0.4800 - val_auc: 0.7652
Epoch 3/15
2198958/2198958 - 26s - loss: 0.4775 - binary_crossentropy: 0.4767 - auc: 0.7686 - val_loss: 0.4783 - val_binary_crossentropy: 0.4774 - val_auc: 0.7686
Epoch 4/15
2198958/2198958 - 26s - loss: 0.4740 - binary_crossentropy: 0.4730 - auc: 0.7734 - val_loss: 0.4769 - val_binary_crossentropy: 0.4758 - val_auc: 0.7708
Epoch 5/15
2198958/2198958 - 26s - loss: 0.4717 - binary_crossentropy: 0.4705 - auc: 0.7767 - val_loss: 0.4757 - val_binary_crossentropy: 0.4745 - val_auc: 0.7724
Epoch 6/15
2198958/2198958 - 26s - loss: 0.4696 - binary_crossentropy: 0.4682 - auc: 0.7795 - val_loss: 0.4754 - val_binary_crossent

## xDeepFM
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/xDeepFM.png" alt="drawing" width="900"/>

In [42]:
xdeep_fm_model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [43]:
all_metrics['xDeepFM'] = test_model(xdeep_fm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='xDeepFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 38s - loss: 0.5033 - binary_crossentropy: 0.5030 - auc: 0.7281 - val_loss: 0.4828 - val_binary_crossentropy: 0.4824 - val_auc: 0.7617
Epoch 2/15
2198958/2198958 - 36s - loss: 0.4790 - binary_crossentropy: 0.4785 - auc: 0.7660 - val_loss: 0.4782 - val_binary_crossentropy: 0.4777 - val_auc: 0.7680
Epoch 3/15
2198958/2198958 - 36s - loss: 0.4738 - binary_crossentropy: 0.4731 - auc: 0.7731 - val_loss: 0.4762 - val_binary_crossentropy: 0.4754 - val_auc: 0.7721
Epoch 4/15
2198958/2198958 - 36s - loss: 0.4700 - binary_crossentropy: 0.4691 - auc: 0.7782 - val_loss: 0.4741 - val_binary_crossentropy: 0.4732 - val_auc: 0.7744
Epoch 5/15
2198958/2198958 - 36s - loss: 0.4669 - binary_crossentropy: 0.4658 - auc: 0.7824 - val_loss: 0.4730 - val_binary_crossentropy: 0.4718 - val_auc: 0.7757
Epoch 6/15
2198958/2198958 - 36s - loss: 0.4644 - binary_crossentropy: 0.4631 - auc: 0.7856 - val_loss: 0.4725 - val_binary_crossent

## AutoInt(Automatic Feature Interaction)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/AutoInt.png" alt="drawing" width="900"/>

In [44]:
auto_int_model = AutoInt(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [45]:
all_metrics['AutoInt'] = test_model(auto_int_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='AutoInt')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 41s - loss: 0.5055 - binary_crossentropy: 0.5052 - auc: 0.7245 - val_loss: 0.4834 - val_binary_crossentropy: 0.4830 - val_auc: 0.7608
Epoch 2/15
2198958/2198958 - 40s - loss: 0.4798 - binary_crossentropy: 0.4793 - auc: 0.7649 - val_loss: 0.4787 - val_binary_crossentropy: 0.4781 - val_auc: 0.7677
Epoch 3/15
2198958/2198958 - 40s - loss: 0.4747 - binary_crossentropy: 0.4739 - auc: 0.7721 - val_loss: 0.4761 - val_binary_crossentropy: 0.4753 - val_auc: 0.7713
Epoch 4/15
2198958/2198958 - 40s - loss: 0.4712 - binary_crossentropy: 0.4703 - auc: 0.7768 - val_loss: 0.4747 - val_binary_crossentropy: 0.4737 - val_auc: 0.7734
Epoch 5/15
2198958/2198958 - 40s - loss: 0.4682 - binary_crossentropy: 0.4671 - auc: 0.7808 - val_loss: 0.4738 - val_binary_crossentropy: 0.4726 - val_auc: 0.7751
Epoch 6/15
2198958/2198958 - 40s - loss: 0.4663 - binary_crossentropy: 0.4650 - auc: 0.7834 - val_loss: 0.4735 - val_binary_crossent

## FiBiNET(Feature Importance and Bilinear feature Interaction NETwork)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/FiBiNET.png" alt="drawing" width="900"/>

In [46]:
fibinet_interaction_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='interaction', dnn_dropout=0.5)

In [47]:
all_metrics['FiBiNET-inter'] = test_model(fibinet_interaction_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-inter')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 62s - loss: 0.5118 - binary_crossentropy: 0.5114 - auc: 0.7136 - val_loss: 0.4829 - val_binary_crossentropy: 0.4823 - val_auc: 0.7623
Epoch 2/15
2198958/2198958 - 45s - loss: 0.4779 - binary_crossentropy: 0.4771 - auc: 0.7684 - val_loss: 0.4759 - val_binary_crossentropy: 0.4749 - val_auc: 0.7720
Epoch 3/15
2198958/2198958 - 46s - loss: 0.4706 - binary_crossentropy: 0.4696 - auc: 0.7784 - val_loss: 0.4739 - val_binary_crossentropy: 0.4727 - val_auc: 0.7752
Epoch 4/15
2198958/2198958 - 46s - loss: 0.4660 - binary_crossentropy: 0.4647 - auc: 0.7845 - val_loss: 0.4726 - val_binary_crossentropy: 0.4712 - val_auc: 0.7768
Epoch 5/15
2198958/2198958 - 46s - loss: 0.4626 - binary_crossentropy: 0.4611 - auc: 0.7889 - val_loss: 0.4726 - val_binary_crossentropy: 0.4709 - val_auc: 0.7771
Epoch 6/15
2198958/2198958 - 46s - loss: 0.4600 - binary_crossentropy: 0.4582 - auc: 0.7922 - val_loss: 0.4735 - val_binary_crossent

In [48]:
fibinet_all_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='all', dnn_dropout=0.5)

In [49]:
all_metrics['FiBiNET-all'] = test_model(fibinet_all_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-all')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 39s - loss: 0.5118 - binary_crossentropy: 0.5115 - auc: 0.7138 - val_loss: 0.4837 - val_binary_crossentropy: 0.4831 - val_auc: 0.7614
Epoch 2/15
2198958/2198958 - 35s - loss: 0.4786 - binary_crossentropy: 0.4778 - auc: 0.7674 - val_loss: 0.4765 - val_binary_crossentropy: 0.4756 - val_auc: 0.7713
Epoch 3/15
2198958/2198958 - 34s - loss: 0.4713 - binary_crossentropy: 0.4702 - auc: 0.7774 - val_loss: 0.4741 - val_binary_crossentropy: 0.4729 - val_auc: 0.7748
Epoch 4/15
2198958/2198958 - 34s - loss: 0.4665 - binary_crossentropy: 0.4652 - auc: 0.7838 - val_loss: 0.4735 - val_binary_crossentropy: 0.4721 - val_auc: 0.7761
Epoch 5/15
2198958/2198958 - 34s - loss: 0.4631 - binary_crossentropy: 0.4615 - auc: 0.7883 - val_loss: 0.4733 - val_binary_crossentropy: 0.4717 - val_auc: 0.7764
Epoch 6/15
2198958/2198958 - 34s - loss: 0.4602 - binary_crossentropy: 0.4585 - auc: 0.7919 - val_loss: 0.4733 - val_binary_crossent

In [50]:
fibinet_each_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='each', dnn_dropout=0.5)

In [51]:
all_metrics['FiBiNET-each'] = test_model(fibinet_each_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-each')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 39s - loss: 0.5119 - binary_crossentropy: 0.5115 - auc: 0.7135 - val_loss: 0.4833 - val_binary_crossentropy: 0.4827 - val_auc: 0.7619
Epoch 2/15
2198958/2198958 - 34s - loss: 0.4783 - binary_crossentropy: 0.4775 - auc: 0.7679 - val_loss: 0.4762 - val_binary_crossentropy: 0.4753 - val_auc: 0.7716
Epoch 3/15
2198958/2198958 - 34s - loss: 0.4708 - binary_crossentropy: 0.4697 - auc: 0.7781 - val_loss: 0.4739 - val_binary_crossentropy: 0.4727 - val_auc: 0.7749
Epoch 4/15
2198958/2198958 - 34s - loss: 0.4662 - binary_crossentropy: 0.4649 - auc: 0.7843 - val_loss: 0.4730 - val_binary_crossentropy: 0.4716 - val_auc: 0.7762
Epoch 5/15
2198958/2198958 - 34s - loss: 0.4627 - binary_crossentropy: 0.4612 - auc: 0.7888 - val_loss: 0.4730 - val_binary_crossentropy: 0.4713 - val_auc: 0.7766
Epoch 6/15
2198958/2198958 - 34s - loss: 0.4602 - binary_crossentropy: 0.4584 - auc: 0.7921 - val_loss: 0.4735 - val_binary_crossent

# Результаты

In [52]:
get_metrics(all_metrics)

,CCPM,PNN,WDL,DeepFM,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-inter,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,,
roc-auc,0.751845,0.774839,0.775921,0.776180,0.757306,0.771769,0.751116,0.776207,0.778106,0.777248,0.777974,0.777417,0.777776
log-loss,0.488265,0.472561,0.471485,0.471306,0.484541,0.474659,0.488973,0.471184,0.469598,0.470147,0.469663,0.470280,0.469772


In [53]:
get_ate(all_metrics, control_name='DeepFM')

,CCPM,PNN,WDL,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-inter,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,
roc-auc,-2.433515,-0.134030,-0.025913,-1.887416,-0.441089,-2.506413,0.002770,0.192624,0.106854,0.179450,0.123769,0.159588
log-loss,1.695883,0.125482,0.017977,1.323562,0.335313,1.766705,-0.012172,-0.170785,-0.115903,-0.164304,-0.102555,-0.153380


In [54]:
get_ate(all_metrics, control_name='FiBiNET-inter')

,CCPM,PNN,WDL,DeepFM,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,
roc-auc,-2.612964,-0.313480,-0.205362,-0.179450,-2.066865,-0.620538,-2.685862,-0.176679,0.013174,-0.072596,-0.055681,-0.019861
log-loss,1.860188,0.289787,0.182282,0.164304,1.487867,0.499618,1.931010,0.152132,-0.006480,0.048402,0.061749,0.010925
